In [14]:
# prerequisites
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

bs = 100
# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

In [15]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        
        # encoder part
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc31 = nn.Linear(h_dim2, z_dim)
        self.fc32 = nn.Linear(h_dim2, z_dim)
        # decoder part
        self.fc4 = nn.Linear(z_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)
        
    def encoder(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc31(h), self.fc32(h) # mu, log_var
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample
        
    def decoder(self, z):
        h = F.relu(self.fc4(z))
        h = F.relu(self.fc5(h))
        return F.sigmoid(self.fc6(h)) 
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 784))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

# build model
vae = VAE(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=2)
vae.cpu()
# if torch.cuda.is_available():
#     vae.cuda()

VAE(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc31): Linear(in_features=256, out_features=2, bias=True)
  (fc32): Linear(in_features=256, out_features=2, bias=True)
  (fc4): Linear(in_features=2, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=784, bias=True)
)

In [16]:
vae

VAE(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc31): Linear(in_features=256, out_features=2, bias=True)
  (fc32): Linear(in_features=256, out_features=2, bias=True)
  (fc4): Linear(in_features=2, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=784, bias=True)
)

In [17]:
optimizer = optim.Adam(vae.parameters())
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

In [23]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        #data = data.cuda()
        data = data.cpu()
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [24]:
def test():
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            #data = data.cuda()
            data = data.cpu()
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [25]:
for epoch in range(1, 51):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 543.514453
Train Epoch: 1 [10000/60000 (17%)]	Loss: 185.244961
Train Epoch: 1 [20000/60000 (33%)]	Loss: 162.233604
Train Epoch: 1 [30000/60000 (50%)]	Loss: 175.613574
Train Epoch: 1 [40000/60000 (67%)]	Loss: 163.437715
Train Epoch: 1 [50000/60000 (83%)]	Loss: 158.294814
====> Epoch: 1 Average loss: 177.4350
====> Test set loss: 161.4401
Train Epoch: 2 [0/60000 (0%)]	Loss: 165.477715
Train Epoch: 2 [10000/60000 (17%)]	Loss: 154.977637
Train Epoch: 2 [20000/60000 (33%)]	Loss: 159.362559
Train Epoch: 2 [30000/60000 (50%)]	Loss: 148.159238
Train Epoch: 2 [40000/60000 (67%)]	Loss: 156.305469
Train Epoch: 2 [50000/60000 (83%)]	Loss: 159.765352
====> Epoch: 2 Average loss: 157.5057
====> Test set loss: 155.0734
Train Epoch: 3 [0/60000 (0%)]	Loss: 154.650254
Train Epoch: 3 [10000/60000 (17%)]	Loss: 157.232295
Train Epoch: 3 [20000/60000 (33%)]	Loss: 162.249395
Train Epoch: 3 [30000/60000 (50%)]	Loss: 149.327012
Train Epoch: 3 [40000/60000 (67%)]	Loss: 156.28

In [29]:
with torch.no_grad():
    # z = torch.randn(64, 2).cuda()
    # sample = vae.decoder(z).cuda()
    z = torch.randn(64, 2).cpu()
    sample = vae.decoder(z).cpu()
    print(sample)
    
    save_image(sample.view(64, 1, 28, 28), './samples/sample_' + '.png')

tensor([[3.7314e-11, 9.8369e-11, 4.6975e-11,  ..., 6.6764e-11, 4.2264e-11,
         4.6756e-11],
        [1.2238e-14, 1.2214e-13, 3.6733e-14,  ..., 2.6934e-14, 1.8585e-13,
         9.6497e-14],
        [8.2317e-12, 1.5964e-11, 1.3177e-11,  ..., 8.6355e-12, 1.4051e-11,
         1.3023e-11],
        ...,
        [4.8957e-10, 4.8266e-10, 5.8374e-10,  ..., 5.0972e-10, 4.8289e-10,
         5.5326e-10],
        [7.0450e-11, 5.7470e-11, 6.0710e-11,  ..., 7.0644e-11, 8.0123e-11,
         8.2901e-11],
        [3.6943e-13, 2.6528e-13, 3.9976e-13,  ..., 2.5179e-13, 2.6314e-13,
         4.1184e-13]])
